#### Advanced include and exclude

The `dict`, `json`, and `copy` methods support `include` and `exclude` arguments which can either be sets or dictionaries. This allows nested selection of which fields to export.

In [1]:
import datetime
from typing import List
from pydantic import BaseModel, Field, SecretStr

In [2]:
class User(BaseModel):
    id: int
    username: str
    password: SecretStr

In [3]:
class Transaction(BaseModel):
    id: str
    user: User
    value: int

In [4]:
t = Transaction(
    id="1234567890",
    user=User(
        id=42,
        username="JohnDoe",
        password="hashedpassword",
    ),
    value=9876543210,
)

In [5]:
# using a set:
print(f"{t.dict(exclude={'user', 'value'}) = }")

t.dict(exclude={'user', 'value'}) = {'id': '1234567890'}


In [6]:
# using a dict:
print(f"{t.dict(exclude={'user': {'username', 'password'}, 'value': True}) = }")

t.dict(exclude={'user': {'username', 'password'}, 'value': True}) = {'id': '1234567890', 'user': {'id': 42}}


In [7]:
print(f"{t.dict(include={'id': True, 'user': {'id'}}) = }")

t.dict(include={'id': True, 'user': {'id'}}) = {'id': '1234567890', 'user': {'id': 42}}


The `True` indicates that we want to exclude or include an entire key, just as if we included it in a set. Of course, the same can be done at any depth level.

Special care must be taken when including or excluding fields from a list or tuple of submodels or dictionaries. In this scenario, `dict` and related methods expect integer keys for element-wise inclusion or exclusion. To exclude a field from **every** member of a list or tuple, the dictionary key `"__all__"` can be used.

In [8]:
class Country(BaseModel):
    name: str
    phone_code: int

In [9]:
class Address(BaseModel):
    post_code: int
    country: Country

In [10]:
class CardDetails(BaseModel):
    number: SecretStr
    expires: datetime.date

In [11]:
class Hobby(BaseModel):
    name: str
    info: str

In [12]:
class ComplexUser(BaseModel):
    first_name: str
    second_name: str
    address: Address
    card_details: CardDetails
    hobbies: List[Hobby]

In [13]:
user = ComplexUser(
    first_name="John",
    second_name="Doe",
    address=Address(
        post_code=123456,
        country=Country(
            name="USA",
            phone_code=1
        )
    ),
    card_details=CardDetails(
        number=4212934504460000,
        expires=datetime.date(2020, 5, 1)
    ),
    hobbies=[
        Hobby(name="Programming", info="Writing code and stuff"),
        Hobby(name="Gaming", info="Hell Yeah!!!"),
    ],
)

In [14]:
exclude_keys = {
    "second_name": True,
    "address": {"post_code": True, "country": {"phone_code"}},
    "card_details": True,
    # You can exclude fields from specific members of a tuple/list by index:
    "hobbies": {-1: {"info"}},
}

In [15]:
include_keys = {
    "first_name": True,
    "address": {"country": {"name"}},
    "hobbies": {0: True, -1: {"name"}},
}

In [16]:
print(f"{user.dict(include=include_keys) = }")
print(f"{user.dict(exclude={'hobbies': {'__all__': {'info'}}}) = }")

user.dict(include=include_keys) = {'first_name': 'John', 'address': {'country': {'name': 'USA'}}, 'hobbies': [{'name': 'Programming', 'info': 'Writing code and stuff'}, {'name': 'Gaming'}]}
user.dict(exclude={'hobbies': {'__all__': {'info'}}}) = {'first_name': 'John', 'second_name': 'Doe', 'address': {'post_code': 123456, 'country': {'name': 'USA', 'phone_code': 1}}, 'card_details': {'number': SecretStr('**********'), 'expires': datetime.date(2020, 5, 1)}, 'hobbies': [{'name': 'Programming'}, {'name': 'Gaming'}]}


The same holds for the `json` and `copy` methods.

##### Model and field level include and exclude

In addition to the explicit arguments `exclude` and `include` passed to `dict`, `json` and `copy` methods, we can also pass the `include/exclude` arguments directly to the `Field` constructor or the equivalent `field` entry in the models `Config` class.

In [17]:
class UserWithConfig(BaseModel):
    id: int
    username: str
    password: SecretStr = Field(..., exclude=True)

In [18]:
class TransactionWithConfig(BaseModel):
    id: str
    user: UserWithConfig = Field(..., exclude={"username"})
    value: int

    class Config:
        fields = {"value": {"exclude": True}}

In [19]:
t = TransactionWithConfig(
    id="1234567890",
    user=UserWithConfig(
        id=42,
        username="JohnDoe",
        password="hashedpassword",
    ),
    value=9876543210,
)
print(f"{t.dict() = }")

t.dict() = {'id': '1234567890', 'user': {'id': 42}}


In the case where multiple strategies are used, `exclude/include` fields are merged according to the following rules:

* First, model config level settings (via `"fields"` entry) are merged per field with the field constructor settings (i.e. `Field(..., exclude=True)`), with the field constructor taking priority.

* The resulting settings are merged per class with the explicit settings on `dict`, `json`, `copy` calls with the explicit settings taking priority.

Note that while merging settings, `exclude` entries are merged by computing the "union" of keys, while `include` entries are merged by computing the "intersection" of keys.

The resulting merged exclude settings:

In [20]:
class UserMergedExclude(BaseModel):
    id: int
    username: str  # overridden by explicit exclude
    password: SecretStr = Field(exclude=True)

In [21]:
class TransactionMergedExclude(BaseModel):
    id: str
    user: UserMergedExclude
    value: int

In [22]:
t = TransactionMergedExclude(
    id="1234567890",
    user=UserMergedExclude(
        id=42,
        username="JohnDoe",
        password="hashedpassword",
    ),
    value=9876543210,
)
print(f"{t.dict(exclude={'value': True, 'user': {'username'}}) = }")

t.dict(exclude={'value': True, 'user': {'username'}}) = {'id': '1234567890', 'user': {'id': 42}}


are the same as using merged include settings as follows:

In [23]:
class UserMergedInclude(BaseModel):
    id: int = Field(..., include=True)
    username: str = Field(..., include=True)  # overridden by explicit include
    password: SecretStr

In [24]:
class TransactionMergedInclude(BaseModel):
    id: str
    user: UserMergedInclude
    value: int

In [25]:
t = TransactionMergedInclude(
    id="1234567890",
    user=UserMergedInclude(
        id=42,
        username="JohnDoe",
        password="hashedpassword",
    ),
    value=9876543210,
)
print(f"{t.dict(include={'id': True, 'user': {'id'}}) = }")

t.dict(include={'id': True, 'user': {'id'}}) = {'id': '1234567890', 'user': {'id': 42}}
